In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
chat = ChatOpenAI(temperature=0.1)


In [ ]:
# b = chat.predict("How many planets are there?")
# b

template = PromptTemplate.from_template("{country_a}와 {country_b}사이의 거리는 어떻게되나요?")
prompt = template.format(country_a="맥시코", country_b="인도")
chat.predict(prompt)

In [15]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리 전문가입니다. 그리고 당신은 {language}로만 대답을 합니다."),
    ("ai", "안녕하세요 제 이름은 {name}입니다."),
    ("human", "{country_a}와 {country_b}사이의 거리는 어떻게되나요? 그리고 당신의 이름은 무었입니까?")
])
prompt = template.format_messages(language="일본어", name="길동", country_a="일본", country_b="한국")
chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 일본과 한국 사이의 거리는 대략 900km 정도입니다. 제 이름은 길동입니다.')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse(" a ,  b ,  c ,   d")

['a', 'b', 'c', 'd']

In [16]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do Not reply with anything else."),
    ("human", "{question}")
])
# prompt = template.format_messages(
#     max_items=10,
#     question="What are the planets"
# )
prompt = template.format_messages(
    max_items=10,
    question="What are the colors"
)
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)


['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [17]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question":"What are the pokemons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [21]:
chef_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisins with easy to find ingredients."),
        ("human","I want to cook {cuisine} food"),
    ]
)
chef_chain = chef_prompt | chat
# chef_chain.invoke({
#     "cuisine":"korean"
# })

In [24]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find a alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat



In [25]:
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
    "cuisine":"korean"
})

AIMessage(content="Great choice! Bibimbap is a delicious Korean dish that can easily be made vegetarian. Here's how you can replace the meat in the recipe:\n\nIngredients:\n- 2 cups cooked rice\n- 1 cup firm tofu, thinly sliced\n- 1 cup julienned carrots\n- 1 cup spinach\n- 1 cup bean sprouts\n- 1 cup sliced mushrooms\n- 4 eggs\n- 4 tablespoons gochujang (Korean chili paste)\n- 2 tablespoons soy sauce\n- 2 tablespoons sesame oil\n- 2 cloves garlic, minced\n- Salt and pepper to taste\n- Vegetable oil for cooking\n\nInstructions:\n1. Marinate the tofu in a mixture of soy sauce, minced garlic, sesame oil, salt, and pepper. Let it sit for at least 15 minutes.\n2. Heat a tablespoon of vegetable oil in a pan over medium heat. Add the marinated tofu and cook until browned and cooked through. Set aside.\n3. In the same pan, add another tablespoon of vegetable oil and sauté the carrots, mushrooms, and bean sprouts separately until tender. Season each with a pinch of salt and pepper. Set aside.\